In [1]:
__author__ = "Yasaman Emami"
__email__ = ['emami.yasamann@gmail.com','yasaman.emami@sjsu.edu']

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Importing Libraries

In [3]:
# !pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.4.1/en_core_web_sm-3.4.1-py3-none-any.whl
# !pip3 install spacy
# !pip3 install contractions


In [4]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 110 kB 29.6 MB/s 
     |████████████████████████████████| 287 kB 56.5 MB/s 


In [5]:
import spacy
import numpy as np
import random
from pickle import dump,load
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding, Dropout, GRU, Activation, Bidirectional
import contractions
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop 
from google.colab import drive
from nltk.tokenize.treebank import TreebankWordDetokenizer
import gensim
import random
from keras.layers.core import Dense, SpatialDropout1D

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [6]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [7]:
filepath = "/content/gdrive/MyDrive/HW5/data.txt"

with open(filepath) as f:
    text = f.read()

# Preprocessing

## 1.c. Expand Contaction

In [8]:
text = contractions.fix(text)


## 1.b. Convert to lowercase

In [10]:
text = text.lower()

In [11]:
# text

## 1.e. Stemming

In [12]:
porter_stemmer = PorterStemmer()
text = porter_stemmer.stem(text)

## 1.e. Lemmatizing

In [13]:
wordnet_lemmatizer = WordNetLemmatizer()
text = wordnet_lemmatizer.lemmatize(text)

## 1.a. Tokenizing

In [14]:
spcay_tokenizer = spacy.load("en_core_web_sm")
spcay_tokenizer = spcay_tokenizer(text)

In [15]:
# spcay_tokenizer

## 1.d. remove punctuation

In [16]:
tokens = [t.text.lower() for t in spcay_tokenizer if t.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [19]:
# take first 45 words for x seq and 46th word as y
seq_len = 46
seq = []
for i in range(seq_len, len(tokens)):
    each_seq = tokens[i-seq_len:i]
    seq.append(each_seq)

In [20]:
" ".join(seq[0])

'call me ishmael some years ago never mind how long precisely having little or no money in my purse and nothing particular to interest me on shore i thought i would sail about a little and see the watery part of the world it is a'

In [21]:
len(seq)

11303

In [22]:
max=int(len(seq)*.75)
seq=seq[:max]

In [23]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(seq)

In [24]:
vocabulary_size = tokenizer.word_counts

In [25]:
seq_all = tokenizer.texts_to_sequences(seq)

In [26]:
len(seq_all)

8477

In [27]:
#token indices
word_index = tokenizer.word_index

In [28]:
seq_all = np.array(seq_all) # converting to numpy array

X = seq_all[:,:-1]
y = seq_all[:,-1]

In [29]:
y.shape

(8477,)

In [30]:
voc_size = len(tokenizer.word_counts) # number of unique words
y = to_categorical(y, num_classes= voc_size+1 )

In [31]:
maxlen = 200
X_train_pad = pad_sequences(X, padding='post', maxlen=maxlen)

In [32]:
X.shape, y.shape

((8477, 45), (8477, 2245))

In [33]:
seq_len = X.shape[1]

In [34]:
seq_len

45

# 2.Text Generation model with keras Embedding layer

In [85]:
def create_model(vocabulary_size, seq_len):
    model = Sequential(name="keras-embedding")
    model.add(Embedding(voc_size, 25, input_length=seq_len))
    model.add(LSTM(150, return_sequences=True))
    model.add(LSTM(150))
    model.add(Dense(150, activation='relu'))

    model.add(Dense(vocabulary_size, activation='softmax'))
    
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
   
    model.summary()
    
    return model


In [36]:
k_model = create_model(voc_size+1, seq_len)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 45, 25)            56100     
                                                                 
 lstm (LSTM)                 (None, 45, 150)           105600    
                                                                 
 lstm_1 (LSTM)               (None, 150)               180600    
                                                                 
 dense (Dense)               (None, 150)               22650     
                                                                 
 dense_1 (Dense)             (None, 2245)              338995    
                                                                 
Total params: 703,945
Trainable params: 703,945
Non-trainable params: 0
_________________________________________________________________


In [37]:
k_model.fit(X, y, batch_size=512, epochs=300,verbose=1)

Epoch 1/300
17/17 [==============================] - 16s 33ms/step - loss: 7.3704 - accuracy: 0.0500
Epoch 2/300
17/17 [==============================] - 0s 27ms/step - loss: 6.4335 - accuracy: 0.0512
Epoch 3/300
17/17 [==============================] - 0s 27ms/step - loss: 6.2931 - accuracy: 0.0552
Epoch 4/300
17/17 [==============================] - 0s 28ms/step - loss: 6.2576 - accuracy: 0.0552
Epoch 5/300
17/17 [==============================] - 0s 27ms/step - loss: 6.2483 - accuracy: 0.0552
Epoch 6/300
17/17 [==============================] - 0s 28ms/step - loss: 6.2424 - accuracy: 0.0552
Epoch 7/300
17/17 [==============================] - 0s 27ms/step - loss: 6.2400 - accuracy: 0.0552
Epoch 8/300
17/17 [==============================] - 0s 28ms/step - loss: 6.2395 - accuracy: 0.0552
Epoch 9/300
17/17 [==============================] - 0s 28ms/step - loss: 6.2395 - accuracy: 0.0552
Epoch 10/300
17/17 [==============================] - 0s 28ms/step - loss: 6.2374 - accuracy: 0.055


# 3. Text Generation with Glove word Embedding


In [39]:
## Glove
glove_dir = '/content/gdrive/MyDrive/HW5/glove.6B.100d.txt'

embedings = {}
with open(glove_dir) as f:
    data = f.readlines()
    
embeddings = {}
for line in data:
    word = line.split()[0]
    embeddings[word] = np.asarray(line.split()[1:],dtype='float32')

In [40]:
max_len = 45
embedding_dim = 100
xtrain = pad_sequences(X,maxlen=max_len)

In [41]:
word_index = tokenizer.word_index
max_features = len(word_index)
embedding_matrix = np.zeros((len(vocabulary_size)+1,embedding_dim))

for word, i in word_index.items():
    if i < max_features:
        
        embedding_vector = embeddings.get(word)
        if embedding_vector is not None:
            # we found the word - add that words vector to the matrix
            embedding_matrix[i] = embedding_vector



In [42]:
embedding_matrix.shape

(2245, 100)

In [43]:
len(vocabulary_size)

2244

In [44]:
len(word_index)

2244

In [83]:
def glove_model():
   
    model = Sequential(name="Glove")

    # model.add(Embedding(len(X),
    #                     100,
    #                     input_length=max_len))

    model.add(Embedding(len(vocabulary_size)+1 , 100,
                                weights = [embedding_matrix],
                                input_length = 45 , 
                                trainable = False))
    # # model.add(Dense(100))
    # model.add(Dense(16,activation='relu'))
    # # model.add(Dropout(0.50))
    # # model.add(Dense(16,activation='relu'))
    # # model.add(Dropout(0.20))


    model.add(LSTM(10,return_sequences=True))
    model.add(LSTM(10))# model.add(LSTM(10))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(1024, activation='relu'))

    model.add(Dense(len(vocabulary_size)+1, activation='softmax'))

    model.summary()
    opt = RMSprop(lr=0.001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [84]:
g_model = glove_model()
g_model.fit(xtrain, y, epochs=300, batch_size=256, verbose=1)


Model: "Glove"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 45, 100)           224500    
                                                                 
 lstm_16 (LSTM)              (None, 45, 10)            4440      
                                                                 
 lstm_17 (LSTM)              (None, 10)                840       
                                                                 
 dense_18 (Dense)            (None, 256)               2816      
                                                                 
 dense_19 (Dense)            (None, 512)               131584    
                                                                 
 dense_20 (Dense)            (None, 1024)              525312    
                                                                 
 dense_21 (Dense)            (None, 2245)              230112

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


34/34 [==============================] - 4s 14ms/step - loss: 6.6799 - accuracy: 0.0524
Epoch 2/300
34/34 [==============================] - 0s 11ms/step - loss: 6.3208 - accuracy: 0.0552
Epoch 3/300
34/34 [==============================] - 0s 10ms/step - loss: 6.2968 - accuracy: 0.0552
Epoch 4/300
34/34 [==============================] - 0s 10ms/step - loss: 6.2905 - accuracy: 0.0552
Epoch 5/300
34/34 [==============================] - 0s 10ms/step - loss: 6.2883 - accuracy: 0.0552
Epoch 6/300
34/34 [==============================] - 0s 11ms/step - loss: 6.2742 - accuracy: 0.0552
Epoch 7/300
34/34 [==============================] - 0s 10ms/step - loss: 6.2718 - accuracy: 0.0552
Epoch 8/300
34/34 [==============================] - 0s 10ms/step - loss: 6.2699 - accuracy: 0.0552
Epoch 9/300
34/34 [==============================] - 0s 11ms/step - loss: 6.2584 - accuracy: 0.0552
Epoch 10/300
34/34 [==============================] - 0s 10ms/step - loss: 6.2590 - accuracy: 0.0552
Epoch 11/30

# 4. Text Generation with word2vec word Embediing

In [48]:
word_model =gensim.models.Word2Vec(size=100, window=5, min_count=1, workers=8)
word_model.build_vocab(seq)
w2v_weights = word_model.wv.vectors
vocab_size, embedding_size = w2v_weights.shape
# vocab_size, embedding_size = pretrained_weights.shape
print("Vocabulary Size: {} - Embedding Dim: {}".format(vocab_size, embedding_size))

Vocabulary Size: 2244 - Embedding Dim: 100


In [49]:
X.shape

(8477, 45)

In [50]:
max_sentence_len = 46


def word2idx(word):
  print(word)
  return word_model.wv.vocab[word].index
def idx2word(idx):
  
  return word_model.wv.index2word[idx]


embedding_matrix = np.zeros((vocab_size+1, 100))
for word, i in tokenizer.word_index.items():
  if word in word_model.wv:
    # weights of the words from word2vec weights
    embedding_matrix[i] = word_model.wv[word]
print(embedding_matrix.shape)


(2245, 100)


In [90]:
def w2v_model():
  model = Sequential(name="w2v-textgeneration")
  model.add(Embedding(input_dim=vocab_size+1, output_dim=embedding_size, weights=[embedding_matrix], input_length=45))
  model.add(LSTM(units=embedding_size))
  model.add(Dense(units=vocab_size//8))
  model.add(Dense(units=vocab_size//4))
  model.add(Dense(units=vocab_size//2))
  model.add(Dense(len(vocabulary_size)+1, activation='softmax'))
  model.summary()
  opt = RMSprop(lr=0.001)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics = ['accuracy'])
  return model


In [91]:
word_model = w2v_model()
word_model.fit(X, y, epochs=300, batch_size=256, verbose=1)

Model: "w2v-textgeneration"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 45, 100)           224500    
                                                                 
 lstm_19 (LSTM)              (None, 100)               80400     
                                                                 
 dense_26 (Dense)            (None, 280)               28280     
                                                                 
 dense_27 (Dense)            (None, 561)               157641    
                                                                 
 dense_28 (Dense)            (None, 1122)              630564    
                                                                 
 dense_29 (Dense)            (None, 2245)              2521135   
                                                                 
Total params: 3,642,520
Trainable params: 3,642,

Generate text

In [54]:
def generate_text(model, tokenizer, seq_len, seed_text, num_gen_words):
    output_text = []
    input_text = seed_text
    for i in range(num_gen_words):
        encoded_text = tokenizer.texts_to_sequences([input_text])[0]
        pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        pred_word_ind = model.predict(pad_encoded, verbose=0)
        pred_word_ind = np.argmax(pred_word_ind,axis=1)[0]
        pred_word = tokenizer.index_word[pred_word_ind]
        input_text += ' '+pred_word
        output_text.append(pred_word)
    return ' '.join(output_text)

In [62]:
sequences = np.array(seq)

In [76]:
random.seed(50)
random_pick = random.randint(0,len(seq))
random_seed_text = seq[random_pick]


In [77]:
seq_all

array([[2244,   20,  209, ...,    8,   10,    2],
       [  20,  209,   46, ...,   10,    2,   84],
       [ 209,   46,  247, ...,    2,   84,    5],
       ...,
       [  27,   62,   25, ..., 2243,    3,   25],
       [  62,   25,    1, ...,    3,   25,  116],
       [  25,    1,  308, ...,   25,  116,  737]])

In [78]:
seed_text = ' '.join(random_seed_text)

# Keras Model Text Generation


In [87]:
generate_text(k_model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=50)

"coat of tropical tanning the multiply nor was welcome the difference in the bowsprit now i will eats done brown if any cut i please down i please i lighted a candle of the whale 's jaw coming belonging the soles of a bamboozingly story i furiously to planing away"

# Glove Model Text Generation

In [86]:
generate_text(g_model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=50)

'reasonest is boat keep the grand stranger a merchant leviathan mouth the bar looking spraining these the purty of darkness the town proved wherever a darkness implement mixed up the fountain money i is the frost small oh the docks door selling nigh the coach carted quiet capering the floor'

# Word2Vec Model Text Generation

In [75]:
generate_text(word_model,tokenizer,seq_len,seed_text=seed_text,num_gen_words=50)

'stranded where else but from nantucket did those aboriginal whalemen the red men first sally out in canoes to give chase to the leviathan and where but from nantucket too did that first adventurous little sloop put forth partly laden with imported cobblestones so goes the story to throw at'